In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader(file_path=r"D:\Shin\NLP\Corpus\namuwikitext\texts\namuwikitext_20200302-ALL.txt", show_progress=True)

docs = loader.load()

print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[:500])

NotImplementedError: Non-relative patterns are unsupported